# [자연어처리]
# 5주차(5-2). 차원 축소(2)

# 202002961 김현주

# 이제 Colab 사용에 모든 수강생분들이 실행에는 어느정도 익숙해 졌을 거라 생각됩니다!
> ## 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해 보기 바랍니다. (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)





---

## 지난 수업 내용 리뷰 -시작-

## PCA를 이용한 차원 축소



데이터 셋 준비 (우리에게 익숙한 20newsgroups)

In [1]:
from sklearn.datasets import fetch_20newsgroups

#20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

#학습 데이터셋을 가져옴
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
#검증 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

X_train = newsgroups_train.data   #학습 데이터셋
y_train = newsgroups_train.target #학습 데이터셋

X_test = newsgroups_test.data     #검증 데이터셋
y_test = newsgroups_test.target   #검증 데이터셋

전처리
* 토큰화, 불용어처리, 스테밍


In [2]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

RegTok = RegexpTokenizer("[\w']{3,}") # 정규포현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) #영어 불용어를 가져옴
stemmer = PorterStemmer() # 스테머로 포터스테머 사용

def my_tokenizer(text):
    tokens = RegTok.tokenize(text)
    # stopwords 제외
    words = [word for word in tokens if word not in english_stops]
    # portr stemmer 적용
    features = [stemmer.stem(token) for token in words]
    return features

In [4]:
# 약간의 시간(약 30초)이 소요됩니다.
from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf = TfidfVectorizer(tokenizer=my_tokenizer, max_features=2000, min_df=5, max_df=0.5)
# 토큰화 외에 아무것도 적용하지 않은 채로 확인
tfidf = TfidfVectorizer(tokenizer=my_tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
print('Original tfidf matrix shape:', X_train_tfidf.shape)

Original tfidf matrix shape: (2034, 20085)


In [6]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression() #분류기 선언

# PCA로 차원 축소된 X 데이터 이용
LR_clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.962
#Test set score: 0.761


## 지난 수업 내용 리뷰 -끝-

---

# LSA를 이용한 차원 축소와 의미 파악


* 기존의 DTM이나 DTM에 단어의 중요도에 따른 가중치를 주었던 TF-IDF 행렬은 단어의 의미를 전혀 고려하지 못한다는 단점을 갖고 있었습니다.
* LSA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 갖고 있습니다.

### DTM

In [7]:
corpus = [
    '먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요'
]

# 각 문서를 공백 기준으로 단어로 분리
tokenized_corpus = [doc.split() for doc in corpus]
print(tokenized_corpus)

[['먹고', '싶은', '사과'], ['먹고', '싶은', '바나나'], ['길고', '노란', '바나나', '바나나'], ['저는', '과일이', '좋아요']]


In [8]:
# 모든 단어들의 집합 생성 후 가나다 순으로 정렬
vocabulary = sorted(set(word for doc in tokenized_corpus for word in doc))
print(vocabulary)

# 단어와 인덱스를 맵핑한 딕셔너리 생성
word2idx = {word: idx for idx, word in enumerate(vocabulary)}
print(word2idx)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']
{'과일이': 0, '길고': 1, '노란': 2, '먹고': 3, '바나나': 4, '사과': 5, '싶은': 6, '저는': 7, '좋아요': 8}


In [9]:
# 빈도수 계산
word_frequency = []
for doc in tokenized_corpus:
    frequency = [0] * len(vocabulary)
    for word in doc:
        if word in word2idx:
            frequency[word2idx[word]] += 1
    word_frequency.append(frequency)

for freq in word_frequency:
    print(freq)

[0, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 0, 2, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 1]


### sklearn의 CountVectorizer를 이용한 DTM

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    '먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요'
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
A = vector.fit_transform(corpus).toarray()

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)
print(A)
print(A.shape)

{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
(4, 9)


---

### LSA를 이용한 차원 축소와 성능


In [11]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2000, random_state=7) #압축할 component의 수 지정

X_train_lsa = svd.fit_transform(X_train_tfidf) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
X_test_lsa = svd.transform(X_test_tfidf)

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

LR_clf.fit(X_train_lsa, y_train) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_lsa, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_lsa, y_test)))

LSA Converted X shape: (2034, 2000)
Sum of explained variance ratio: 1.000
#Train set score: 0.962
#Test set score: 0.761


### LSA 벡터 유사도 측정

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

print('#사용된 전체 카테고리:', newsgroups_train.target_names)
print('#첫 문서의 카테고리:', y_train[0])

print('#첫 문서의 내용:', X_train[0])

#변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

print("#Top 10 유사도(lsa):\n", sorted(sim_result[0].round(2), reverse=True)[:10])
sim_index = (-sim_result[0]).argsort()[:10]
print('#Top 10 유사 뉴스의 인덱스(lsa):\n', sim_index)
sim_labels = [y_train[i] for i in sim_index]
print('#Top 10 유사 뉴스의 카테고리(lsa):\n', sim_labels)

#사용된 전체 카테고리: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#첫 문서의 카테고리: 1
#첫 문서의 내용: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Top 10 유사도(lsa):
 [1.0, 0.3, 0.22, 0.21, 0.19, 0.19, 0.19, 0.17, 0.16, 0.16]
#Top 10 유사 뉴스의 인덱스(lsa):
 [   0 1575 1892 1490  501 1290 1013  998 1636 1705]
#Top 10 유사 뉴스의 카테고리(lsa):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### TF-IDF 벡터 유사도 측정

In [13]:
sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

# 유사도가 높은 순으로 정렬
print("#Top 10 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:10])

# 유사도가 높은 순으로 정렬 후, 해당 값의 index를 반환
sim_index = (-sim_result[0]).argsort()[:10]
print('#Top 10 유사 뉴스의 인덱스(tfidf):\n', sim_index)
sim_labels = [y_train[i] for i in sim_index]
print('#Top 10 유사 뉴스의 카테고리(tfidf):\n', sim_labels)

#Top 10 유사도(tfidf):
 [1.0, 0.3, 0.22, 0.21, 0.19, 0.19, 0.19, 0.17, 0.16, 0.16]
#Top 10 유사 뉴스의 인덱스(tfidf):
 [   0 1575 1892 1490  501 1290 1013  998 1636 1705]
#Top 10 유사 뉴스의 카테고리(tfidf):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


---

# **[[실습 5-2-1]]** 유사도를 이용한 영화 추천 시스템 구현하기: TF-IDF 벡터 vs LSA 벡터
영화 소개, 제목, 평점 등이 있는 **'movies_metadata.csv'** 데이터 셋 **(LMS에서 다운로드 후 업로드)**을 이용하여, 특정 영화와 가장 유사한 영화를 찾아 추천해주기

* 영화 소개(overview)에 대한 내용을 각각 TF-IDF 벡터로도 나타내보고, LSA로도 나타내보기
* 유사한 영화에 자기 자신을 포함할 수 있음. 가능한 경우, 이를 따로 처리해도 좋고 처리하지 않아도 됨
* Dataset 출처: https://www.kaggle.com/rounakbanik/the-movies-dataset
<br>
<br>

### [제출 내용]

1.   TF-IDF를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title)를 10개 찾는 코드 작성 (코드 제공)
  - 모든 feature를 다 사용
    
2.   LSA를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title)를 10개 찾는 코드 작성 (코드 작성 필요)
  - **n_components=500**으로 작성
3.   TF-IDF와 LSA의 결과 비교 설명. LSA가 의미를 더 잘 찾는 것 같은지? 자신에게는 TF-IDF와 LSA 중 어떤 것이 더 마음에 드는 방법이었는지 결론을 내려보기
    * 유사도, 영화 소개의 내용 등을 참고하여 비교해 보시오.


### [참고 코드]
* **참고용**으로 사용하시면 됩니다. 더 편한 방법이 있다면 해당 방법으로 작성하셔도 좋습니다.
* 참고 코드는 0번째 영화 & TF-IDF에 대해서만 작성되어 있음
    * 0번째 영화가 아닌 다른 n번째 영화를 선택하여 비교해보아도 좋습니다.
    * **(주의!)** LSA에 대한 코드를 추가 작성해야 합니다. (n_components=500으로 작성)

# 1. [코드 제공] TF-IDF를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title) 10개 찾기

In [14]:
import pandas as pd
df = pd.read_csv('./movies_metadata.csv', encoding='utf-8', low_memory=False)
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [15]:
from sklearn.model_selection import train_test_split

# vote_count 500 이상인 영화 중에서 추천
sampled_df = df[df['vote_count'] >= 500].reset_index(drop=True)
sampled_df.shape

(2056, 24)

In [16]:
# overview에 빈 값이 있는 경우, ''로 채우기
sampled_df['overview'] = sampled_df['overview'].fillna('')

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(sampled_df.overview, sampled_df.title, random_state=0)

tfidf = TfidfVectorizer() # (max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

print("0번째 영화 제목\n", y_train[0])
print("0번째 영화 소개 내용\n", X_train[0])

sim_result = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

print("#Top 10 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:10])
sim_index = (-sim_result[0]).argsort()[:10]

0번째 영화 제목
 Toy Story
0번째 영화 소개 내용
 Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.
#Top 10 유사도(tfidf):
 [1.0, 0.18, 0.17, 0.16, 0.16, 0.15, 0.15, 0.15, 0.14, 0.14]


In [19]:
result_df = pd.DataFrame({
    '유사도': sorted(sim_result[0].round(2), reverse=True)[:10],
    '인덱스': sim_index,
    '제목': [sampled_df.title[i] for i in sim_index],
    '영화 소개': [sampled_df.overview[i] for i in sim_index],
    '평점': [sampled_df.vote_average[i] for i in sim_index]
})

result_df

,유사도,인덱스,제목,영화 소개,평점
0,1.00,0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.7
1,0.18,334,The Prince of Egypt,This is the extraordinary tale of two brothers...,6.9
2,0.17,635,The Matrix Reloaded,Six months after the events depicted in The Ma...,6.7
3,0.16,1163,Planet 51,When Earth astronaut Capt. Chuck Baker arrives...,5.6
4,0.16,826,Corpse Bride,"Set in a 19th-century european village, this s...",7.2
5,0.15,878,Click,A workaholic architect finds a universal remot...,6.0
6,0.15,1416,The Cabin in the Woods,Five college friends spend the weekend at a re...,6.6
7,0.15,512,Memento,Suffering short-term memory loss after a head ...,8.1
8,0.14,1359,50/50,"Inspired by a true story, a comedy centered on...",7.0
9,0.14,625,Irreversible,Events over the course of one traumatic night ...,7.1


In [20]:
X_train_tfidf.shape

(1542, 11269)

In [21]:
print('TF-IDF 기반으로 계산한 결과 가장 추천하는 영화는: ', [sampled_df.title[i] for i in sim_index][1:2])

TF-IDF 기반으로 계산한 결과 가장 추천하는 영화는:  ['The Prince of Egypt']


# 2. [코드 작성 필요] LSA를 이용하여, 영화 소개(overview)의 내용이 가장 유사한 영화(title) 10개 찾기
- **n_components=500**으로 작성

In [22]:
# 코드 작성
# 시작이 어려운 경우, 아래의 뼈대 코드를 참고하여, 4개의 코드 셀을 완성해 보기 바랍니다.

In [23]:
# from sklearn.decomposition import TruncatedSVD

# svd = ... #압축할 component의 수 지정 (n_components=500)

# X_train_lsa = ...

# print('LSA Converted X shape:', X_train_lsa.shape)

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, random_state=7) #압축할 component의 수 지정

X_train_lsa = svd.fit_transform(X_train_tfidf)

print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))

LSA Converted X shape: (1542, 500)
Sum of explained variance ratio: 0.558


In [24]:
#from sklearn.metrics.pairwise import cosine_similarity

# print("0번째 영화 제목\n", y_train[0])
# print("0번째 영화 소개 내용\n", X_train[0])

# sim_result = ...

# print("#Top 10 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:10])
# sim_index = ...

from sklearn.metrics.pairwise import cosine_similarity

print("0번째 영화 제목\n", y_train[0])
print("0번째 영화 소개 내용\n", X_train[0])

sim_result = cosine_similarity([X_train_lsa[0]], X_train_lsa)

print("#Top 10 유사도(tfidf):\n", sorted(sim_result[0].round(2), reverse=True)[:10])
sim_index = (-sim_result[0]).argsort()[:10]

0번째 영화 제목
 Toy Story
0번째 영화 소개 내용
 Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.
#Top 10 유사도(tfidf):
 [1.0, 0.37, 0.36, 0.36, 0.35, 0.33, 0.32, 0.32, 0.31, 0.28]


In [25]:
# result_df = ...

# result_df

result_df = pd.DataFrame({
    '유사도': sorted(sim_result[0].round(2), reverse=True)[:10],
    '인덱스': sim_index,
    '제목': [sampled_df.title[i] for i in sim_index],
    '영화 소개': [sampled_df.overview[i] for i in sim_index],
    '평점': [sampled_df.vote_average[i] for i in sim_index]
})

result_df

,유사도,인덱스,제목,영화 소개,평점
0,1.00,0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.7
1,0.37,334,The Prince of Egypt,This is the extraordinary tale of two brothers...,6.9
2,0.36,635,The Matrix Reloaded,Six months after the events depicted in The Ma...,6.7
3,0.36,1163,Planet 51,When Earth astronaut Capt. Chuck Baker arrives...,5.6
4,0.35,1359,50/50,"Inspired by a true story, a comedy centered on...",7.0
5,0.33,1416,The Cabin in the Woods,Five college friends spend the weekend at a re...,6.6
6,0.32,878,Click,A workaholic architect finds a universal remot...,6.0
7,0.32,512,Memento,Suffering short-term memory loss after a head ...,8.1
8,0.31,826,Corpse Bride,"Set in a 19th-century european village, this s...",7.2
9,0.28,1431,Brave,Brave is set in the mystical Scottish Highland...,6.7


In [26]:
print('LSA 기반으로 계산한 결과 가장 추천하는 영화는: ', [sampled_df.title[i] for i in sim_index][1:2])

LSA 기반으로 계산한 결과 가장 추천하는 영화는:  ['The Prince of Egypt']


# 3. TF-IDF와 LSA의 결과 비교 설명.


TF-IDF의 경우 Top 10의 영화에는 Irreversible가 있는데, LSA의 경우에는 Brave가 있다.그 이유는 LSA는 Truncated SVD를 이용하여 대각원소 가운데 상위 t개만 골라내어 잠재 의미 중 비중이 큰 것부터 차례대로 골라내어 차원 축소를 수행하는데, 이때 TF-IDF의 경우보다 차원이 작아지고 더 적은 공간에 문서들이 위치하게 되기 때문이다.


---

# [파일] -> [다운로드] -> [.ipynb 다운로드]